In [ ]:
import numpy as np
from scipy.stats import maxwell
import matplotlib.pyplot as plt
from scipy import stats 

$$E_{peak} = k_bT$$
$$v_{max} = \sqrt{E_{peak} / m}$$

In [ ]:
k_b = 1.380649e-23 # J/K
T = 20 # K 
m_n = 1.67492749804e-27 # kg
h = 6.62607015e-34 # J / Hz
a = np.sqrt(k_b * T / m_n)
a

In [ ]:
max_v = np.sqrt(2) * a
max_v

In [ ]:
v = np.linspace(300,6000,10000)
pdf_v = maxwell.pdf(v, scale=a)
min_y = np.min(pdf_v)
max_y = np.max(pdf_v)
plt.plot(v, pdf_v, 'r-',alpha=0.6,label='maxwell pdf')
plt.ylim(min_y,max_y)
plt.xlabel('v [m/s]')
plt.ylabel('pdf')
plt.axvline(max_v, linestyle='--')
plt.grid()

$$\lambda = \frac{h}{mv}$$

In [ ]:
wavelength = h / (m_n *v)
plt.plot(v, wavelength)

Original pdf
$$\int f_v dv = 1$$
Todo: find $f_\lambda$ such that
$$\int f_\lambda d\lambda = 1$$
With $\lambda = \frac{h}{m_n v}$
$$\implies d\lambda = \frac{-h}{m_n v ^ 2}dv$$
$$\implies dv = \frac{-m_n v ^ 2}{h}d\lambda$$

In practice, reordering the bounds of the integral will cancel the $-1$ factor.


In [ ]:
print(wavelength)
f_v_to_f_wave = m_n * v**2 / h 
f_v_to_f_wave = h / (m_n * wavelength ** 2)
pdf_wave = pdf_v * f_v_to_f_wave
plt.plot(wavelength * 1e10, pdf_wave, 'r',alpha=0.6,label='maxwell pdf')
# plt.ylim(0,0.0004)
plt.xlabel(r'$\lambda$ [Å]')
plt.ylabel('pdf')
plt.grid()

In [ ]:
np.sum(pdf_wave * 1e-10)

# Direct computation using transformed Maxwell-Boltzmann

In [ ]:
def comp_pdf_wave(wavelength, T):
    a = np.sqrt(k_b * T / m_n)
    return maxwell.pdf(h / (m_n * wavelength), scale=a) * h / (m_n * wavelength ** 2)

w_0 = 2.165e-10
w_1 = 4.321e-10
w_2 = 8e-10
T = 20
center_1 = comp_pdf_wave(w_1,T)
center_2 = comp_pdf_wave(w_2,T)
center_1/center_2

wavelength = np.linspace(0.01e-10, 12e-10, 100000) # m
d_wavelength = wavelength[1] - wavelength[0]
v = h / (m_n * wavelength)
# T is in Kelvin
for T in [20,290]:
    a = np.sqrt(k_b * T / m_n)
    pdf_wave = comp_pdf_wave(wavelength, T)
    plt.plot(wavelength * 1e10, pdf_wave * 1e-10,label=r'$T=' +str(T)+r'$K')
    # plt.ylim(0,0.0004)
    plt.xlabel(r'$\lambda$ [Å]')
    plt.ylabel(r'$f_\lambda$ [Å$^{-1}$]')
# for w in [w_0, w_1, w_2]:
#     plt.axvline(w * 1e10, linestyle='--')
plt.grid()
plt.legend()
plt.savefig("docs/source-spectrum.eps",bbox_inches="tight", pad_inches=0, format='eps')

# Monochromators
$\lambda = 8 Å$ means a helical slot velocity selector rotating at a set velocity.
$\lambda = 4 Å$ means a pyroletic graphite crystal rotated to the correct angle.

[FWHM](https://en.wikipedia.org/wiki/Full_width_at_half_maximum) is a common term when expressing $\Delta\lambda/\lambda$, and the relation to a Gauss is through $\Delta\lambda = 2 \sqrt{2\ln2} \sigma$

For the helical slot velocity selector  $\Delta\lambda \approx 0.1\lambda$, for the PG crystal $\Delta\lambda \approx 0.01\lambda$ 

In [ ]:
def filter(wavelengths, center_wavelength, delta_wavelength):
    sigma = delta_wavelength / (2 * np.sqrt(2 * np.log(2)))
    return np.exp(-(wavelengths - center_wavelength) ** 2 / (2 * sigma**2))
monochrom_options = [(w_1,0.01), (w_2, 0.05), (w_2, 0.1), (w_2, 0.2)]
monochrom_transfer = np.zeros((4, len(wavelength)))

for (i,(w_0, f)) in enumerate(monochrom_options):
    monochrom_transfer[i] = filter(wavelength, w_0, f * w_0)

for transfer in monochrom_transfer:
    plt.plot(wavelength * 1e10, transfer)  
plt.xlabel(r'$\lambda$ [Å]')
plt.ylabel(r'$f_\lambda$')
plt.grid()

In [ ]:
T = 20
pdf_wave = comp_pdf_wave(wavelength, T)
selected = pdf_wave * monochrom_transfer
for selected_i in selected:
    plt.plot(wavelength * 1e10, selected_i)
plt.xlabel(r'$\lambda$ [Å]')
plt.ylabel(r'$f_\lambda$')
plt.grid()

In [ ]:
from scipy import integrate
flux_fraction = integrate.trapezoid(selected, axis=1) * d_wavelength
print(flux_fraction)
flux_fraction[2]/flux_fraction[0]

In [ ]:
C_mc_4321 = flux_fraction[0]
C_mc_8 = flux_fraction[2]
C_pol_ana = 1/4

R = 0.1
d = 6
psi_0 = np.deg2rad(2)
div_factor = (R**2) / ((R + psi_0 * d)**2)

phi_0 = 1000e6 * 0.5
print(phi_0)

for C_mc in [C_mc_4321, C_mc_8]:
    print(div_factor, C_mc, C_pol_ana)
    print(div_factor * C_mc * C_pol_ana)
    est_phi_0 = phi_0 * div_factor * C_mc * C_pol_ana
    print(f"{est_phi_0}/cm^2/s, 10^{round(np.log10(est_phi_0),2)}")
    # print(est_phi_0)
    # print(np.log10(est_phi_0))
    # print(phi_0 * div_factor * C_mc * C_pol_ana)



# Divergence
First, a maximum value is calculated. Next, an approximate divergence distribution is computed

In [ ]:
h_d = 10e-3
b_d = 10e-3

h_b_dist = 5

psi_0 = np.arctan((h_d + b_d)/5)
psi_0 * 1e3

In [ ]:
N = 100000

h_d = 10e-3
b_d = 10e-3
d = 5

def sum_of_two_uniform(s):
    size = b_d + h_d

param_space = [{'low': -b_d/2, 'high': b_d/2}, {'low': -h_d/2, 'high': h_d/2}]

# Generate the random array
combinations = np.zeros((N, 2))

for i, param in enumerate(param_space):
    combinations[:, i] = np.random.uniform(low=param['low'], high=param['high'], size=N)
psi_0 = np.arctan((h_d/2 + b_d/2)/d)
psi_2d = np.arctan(np.sqrt(2) * (h_d/2 + b_d/2)/d)
psi_0, psi_2d

In [ ]:
def trapezoid_pdf(x, a1, b1, a2, b2):
    result = np.zeros_like(x)
    # Rising linear section (triangle)
    mask1 = (x >= (a1 + a2)) & (x < (b1 + a2))
    result[mask1] = (x[mask1] - (a1 + a2)) / ((b1 - a1) * (b2 - a2))
    # Flat section (square)
    mask2 = (x >= (b1 + a2)) & (x < (a1 + b2))
    result[mask2] = 1 / (b1 - a1)
    # Falling linear section (triangle)
    mask3 = (x >= (a1 + b2)) & (x < (b1 + b2))
    result[mask3] = ((b1 + b2) - x[mask3]) / ((b1 - a1) * (b2 - a2))
    return result

# Parameters
a1, b1 = -b_d/2, b_d/2
a2, b2 = -h_d/2, h_d/2

# Range for x values
x = np.linspace(a1 + a2, b1 + b2, 4000)
pdf_values = trapezoid_pdf(x, a1, b1, a2, b2)

# Plot the PDF
# plt.figure(figsize=(12, 6))
plt.plot(x, pdf_values, label='PDF')
# plt.fill_between(x, pdf_values, alpha=0.2)
plt.title('Probability Density Function')
plt.xlabel('x')
plt.ylabel('Density')
plt.legend()
plt.show()

In [ ]:
# x = np.linspace(a1 + a2, b1 + b2, 1000)
X, Y = np.meshgrid(x, x, indexing='ij')

div = np.arctan(np.sqrt(X ** 2 + Y ** 2)/d)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X * 1e3, Y * 1e3, div, cmap='viridis', edgecolor='none')
cbar = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, pad=0.1)
cbar.set_label(r'$I$ [a.u.]', fontsize=12)
ax.set_xlabel(r'$x$ [mm]')
ax.set_ylabel(r'$y$ [mm]')
plt.show()

# plt.plot(x,div * 1e3)
# plt.show()

In [ ]:
# Range for x values
x = np.linspace(a1 + a2, b1 + b2, 4000)
pdf_values = trapezoid_pdf(x, a1, b1, a2, b2)

# x = np.linspace(a1 + a2, b1 + b2, 4000)
X, Y = np.meshgrid(x, x, indexing='ij')
joint_pdf = trapezoid_pdf(X, a1, b1, a2, b2) * trapezoid_pdf(Y, a1, b1, a2, b2)

# cum_pdf = trapezoid_pdf(X, a1, b1, a2, b2) * trapezoid_pdf(Y, a1, b1, a2, b2)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X * 1e3, Y * 1e3, joint_pdf, cmap='viridis', edgecolor='none')
cbar = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, pad=0.1)
cbar.set_label(r'$I$ [a.u.]', fontsize=12)
ax.set_xlabel(r'$x$ [mm]')
ax.set_ylabel(r'$y$ [mm]')
plt.show()

# plt.plot(x,div * 1e3)
# plt.show()

In [ ]:

# cum_pdf = trapezoid_pdf(X, a1, b1, a2, b2) * trapezoid_pdf(Y, a1, b1, a2, b2)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X * 1e3, Y * 1e3, joint_pdf * div, cmap='viridis', edgecolor='none')
cbar = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, pad=0.1)
cbar.set_label(r'$I$ [a.u.]', fontsize=12)
ax.set_xlabel(r'$x$ [mm]')
ax.set_ylabel(r'$y$ [mm]')
plt.show()

# plt.plot(x,div * 1e3)
# plt.show()

In [ ]:
dx = x[1] - x[0]
values = (joint_pdf * div).flatten()
mean_div = np.mean(values) / 160000
print(mean_div)
# # Create the histogram
# plt.figure(figsize=(10, 6))
# plt.hist(values * 1e3, bins=1000, density=True, alpha=0.75, color='blue', edgecolor='black')
# # plt.title('Histogram of Z = f(X, Y)')
# plt.xlabel('Z')
# plt.ylabel('Density')
# plt.show()